## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


TensorFlow 2.x selected.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
        mean = np.mean(X_train, axis=(0, 1, 2, 3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train - mean) / (std + 1e-7)
        X_test = (X_test - mean) / (std + 1e-7) 
        return X_train, X_test,mean, std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder(categories='auto')
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

In [4]:
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3)))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 9s 175us/sample - loss: 1.4644 - accuracy: 0.4843
Epoch 2/100
50000/50000 [==============================] - 4s 81us/sample - loss: 1.0547 - accuracy: 0.6260
Epoch 3/100
50000/50000 [==============================] - 4s 81us/sample - loss: 0.9318 - accuracy: 0.6723
Epoch 4/100
50000/50000 [==============================] - 4s 80us/sample - loss: 0.8527 - accuracy: 0.7003
Epoch 5/100
50000/50000 [==============================] - 4s 81us/sample - loss: 0.7764 - accuracy: 0.7274
Epoch 6/100
50000/50000 [==============================] - 4s 79us/sample - loss: 0.7114 - accuracy: 0.7497
Epoch 7/100
50000/50000 [==============================] - 4s 80us/sample - loss: 0.6405 - accuracy: 0.7762
Epoch 8/100
50000/50000 [==============================] - 4s 79us/sample - loss: 0.5764 - accuracy: 0.7981
Epoch 9/100
50000/50000 [==============================] - 4s 80us/sample - loss: 0.5146 - accuracy: 0.8189
Epoc

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example = (np.zeros(shape=(1,32,32,3)) - mean_train) / (std_train+1e-7) 
classifier.predict(input_example)

array([[1.2976918e-10, 4.5743865e-22, 3.7155709e-13, 9.9999893e-01,
        1.0943432e-06, 0.0000000e+00, 9.1020136e-10, 1.7888926e-12,
        2.0721790e-18, 6.3409397e-20]], dtype=float32)

In [7]:
# 嘗試不同搭法
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(64, (3,3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#GAP
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(units=100, activation='relu'))

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100)

input_example = (np.zeros(shape=(1,32,32,3)) - mean_train) / (std_train+1e-7) 
classifier.predict(input_example)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 5s 97us/sample - loss: 1.5830 - accuracy: 0.4221
Epoch 2/100
50000/50000 [==============================] - 4s 82us/sample - loss: 1.2740 - accuracy: 0.5464
Epoch 3/100
50000/50000 [==============================] - 4s 81us/sample - loss: 1.1470 - accuracy: 0.5908
Epoch 4/100
50000/50000 [==============================] - 4s 82us/sample - loss: 1.0688 - accuracy: 0.6208
Epoch 5/100
50000/50000 [==============================] - 4s 81us/sample - loss: 1.0130 - accuracy: 0.6418
Epoch 6/100
50000/50000 [==============================] - 4s 81us/sample - loss: 0.9722 - accuracy: 0.6583
Epoch 7/100
50000/50000 [==============================] - 4s 81us/sample - loss: 0.9317 - accuracy: 0.6705
Epoch 8/100
50000/50000 [==============================] - 4s 82us/sample - loss: 0.9024 - accuracy: 0.6817
Epoch 9/100
50000/50000 [==============================] - 4s 81us/sample - loss: 0.8741 - accuracy: 0.6890
Epoch

array([[2.2414514e-01, 6.2235252e-08, 2.5117914e-03, 3.4780521e-02,
        6.8707038e-03, 2.8476263e-06, 1.2895263e-02, 2.6983039e-03,
        7.1609509e-01, 2.5378921e-07]], dtype=float32)

In [8]:
# 嘗試不同搭法
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#GAP
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(units=100, activation='relu'))

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100)

input_example = (np.zeros(shape=(1,32,32,3)) - mean_train) / (std_train+1e-7) 
classifier.predict(input_example)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 4s 74us/sample - loss: 1.4681 - accuracy: 0.4714
Epoch 2/100
50000/50000 [==============================] - 3s 60us/sample - loss: 1.1812 - accuracy: 0.5789
Epoch 3/100
50000/50000 [==============================] - 3s 60us/sample - loss: 1.0766 - accuracy: 0.6148
Epoch 4/100
50000/50000 [==============================] - 3s 60us/sample - loss: 1.0071 - accuracy: 0.6437
Epoch 5/100
50000/50000 [==============================] - 3s 59us/sample - loss: 0.9573 - accuracy: 0.6618
Epoch 6/100
50000/50000 [==============================] - 3s 60us/sample - loss: 0.9155 - accuracy: 0.6779
Epoch 7/100
50000/50000 [==============================] - 3s 60us/sample - loss: 0.8792 - accuracy: 0.6895
Epoch 8/100
50000/50000 [==============================] - 3s 60us/sample - loss: 0.8512 - accuracy: 0.7004
Epoch 9/100
50000/50000 [==============================] - 3s 60us/sample - loss: 0.8190 - accuracy: 0.7123
Epoch

array([[1.4483124e-02, 2.8368779e-05, 4.0972441e-02, 3.7808037e-01,
        2.9740152e-01, 9.9509209e-03, 1.6080401e-03, 7.9493917e-04,
        2.5667906e-01, 1.1756422e-06]], dtype=float32)